In [54]:
#importing dependencies and setting app token
import pandas as pd
import requests
from datetime import date, datetime
from sodapy import Socrata
#from config import MyAppToken
MyAppToken = 'GuILMuJLLhVOQ8u9cyXPc56p3'

#checking the day of the month and printing the result, this is used to filter the dataframe later
today = date.today()
day = today.strftime("%d")
print("Day =", day)

Day = 09


In [2]:
client = Socrata("data.cityofchicago.org", MyAppToken, None, None )

#getting () results from the soda api
try:
    results = client.get("ijzp-q8t2", limit=20000)
                         
except requests.exceptions.Timeout:
  print("Timeout occurred")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [78]:
#dropping columns that have no use
clean_columns_df = results_df.drop(columns=['location','x_coordinate', 'y_coordinate','id','beat','district','ward','community_area','fbi_code','updated_on', 'case_number', 'block', 'iucr', 'location_description', 'arrest', ':@computed_region_awaf_s7ux', ':@computed_region_6mkv_f3dw', ':@computed_region_vrxf_vc4k', ':@computed_region_bdys_3d7i', ':@computed_region_43wa_7qmu' , ':@computed_region_rpca_8um6',':@computed_region_d9mm_jgwp',':@computed_region_d3ds_rm58'])
#adding chicago column for organization while grouping
clean_columns_df['city'] = "Chicago"
#formatting datetime to ease readability
clean_columns_df['month'] = pd.to_datetime(clean_columns_df['date']).dt.strftime('%m')
clean_columns_df['date'] = pd.to_datetime(clean_columns_df['date']).dt.strftime('%d')
clean_columns_df.rename(columns = {'date':'day'}, inplace = True)

clean_columns_df.head()

,day,primary_type,description,domestic,year,latitude,longitude,city,month
0,01,THEFT,FROM BUILDING,False,2020,41.946264373,-87.678537261,Chicago,04
1,01,CRIMINAL DAMAGE,TO PROPERTY,True,2020,41.900884118,-87.722026278,Chicago,04
2,01,MOTOR VEHICLE THEFT,AUTOMOBILE,False,2020,42.008077269,-87.690148921,Chicago,04
3,01,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,False,2020,41.895826124,-87.726096446,Chicago,04
4,01,THEFT,$500 AND UNDER,True,2020,41.764779745,-87.67637213,Chicago,04


In [84]:
#reorganizing columns to read easier left to right
clean_columns_df = clean_columns_df[["city",'primary_type','description','domestic', "day", "month", "year", "latitude", "longitude"]]

In [91]:
domestic_crimes = clean_columns_df['domestic'] == True
non_domestic_crimes = clean_columns_df['domestic'] == False
domestic_crimes_df = clean_columns_df[domestic_crimes]
non_domestic_crimes_df = clean_columns_df[non_domestic_crimes]

domestic_crimes_df

,city,primary_type,description,domestic,day,month,year,latitude,longitude
1,Chicago,CRIMINAL DAMAGE,TO PROPERTY,True,01,04,2020,41.900884118,-87.722026278
4,Chicago,THEFT,$500 AND UNDER,True,01,04,2020,41.764779745,-87.67637213
5,Chicago,CRIMINAL DAMAGE,TO PROPERTY,True,01,04,2020,42.01837759,-87.681639928
9,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,01,04,2020,41.720382956,-87.658960167
16,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,01,04,2020,41.802747146,-87.616282132
...,...,...,...,...,...,...,...,...,...
19955,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,23,02,2020,41.735502667,-87.620969222
19966,Chicago,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,True,23,02,2020,41.703783161,-87.605532421
19978,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,23,02,2020,41.962361027,-87.654756322
19988,Chicago,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,True,23,02,2020,41.727313196,-87.668522373


In [90]:
#still have to use day to filter results from last year
non_domestic_crimes_df

,city,primary_type,description,domestic,day,month,year,latitude,longitude
0,Chicago,THEFT,FROM BUILDING,False,01,04,2020,41.946264373,-87.678537261
2,Chicago,MOTOR VEHICLE THEFT,AUTOMOBILE,False,01,04,2020,42.008077269,-87.690148921
3,Chicago,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,False,01,04,2020,41.895826124,-87.726096446
6,Chicago,BURGLARY,FORCIBLE ENTRY,False,01,04,2020,41.872992906,-87.63063219
7,Chicago,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,False,01,04,2020,41.766617035,-87.674212669
...,...,...,...,...,...,...,...,...,...
19995,Chicago,CRIMINAL DAMAGE,TO PROPERTY,False,23,02,2020,41.683044898,-87.642085558
19996,Chicago,BATTERY,AGGRAVATED - HANDGUN,False,23,02,2020,41.876914017,-87.679825838
19997,Chicago,THEFT,OVER $500,False,23,02,2020,41.735777715,-87.663976174
19998,Chicago,CRIMINAL DAMAGE,TO PROPERTY,False,23,02,2020,41.886305173,-87.714566983
